# *btest* initial steps

step-by-step

In [1]:
import argparse
import csv
import itertools
import logging
import os
import sys
import shutil
import time
import math
import random
from scipy.stats import pearsonr
import numpy as np
import pandas as pd
import math
from scipy.stats import spearmanr
from btest import config
from btest import utils
from btest import blockplot


In [4]:
dataX, dataY = utils.readData('~/Documents/omicsEye/btest/demo/X_dataset.txt', '~/Documents/omicsEye/btest/demo/Y_dataset.txt')

In [5]:
dataAll, featuresX, featuresY = utils.dataProcess(dataX, dataY)

In [6]:
dataAll

array([[ 5.,  6.,  5., ...,  1.,  4.,  3.],
       [ 5., nan,  4., ...,  1.,  4.,  3.],
       [ 5.,  5.,  5., ...,  2.,  4.,  4.],
       ...,
       [ 4.,  1.,  1., ...,  4.,  5.,  6.],
       [ 4.,  1.,  1., ...,  4.,  5.,  6.],
       [ 4.,  1.,  1., ...,  4.,  6.,  5.]])

In [7]:
def btest_corr(dataAll, featuresX, featuresY, method='spearman', fdr=0.1):
    iRow = len(dataAll)
    iCol = len(dataAll)
    tests = []
    features = featuresX + featuresY
    for i in list(range(iRow)):
        for j in list(range(iCol)):
            if i<=j:
                X = dataAll[i]
                Y=  dataAll[j]
                nas = np.logical_or(X != X, Y != Y)
                not_na = sum(~nas)
                #X = Y[~nas]
                #Y = Y[~nas]
                #new_X, new_Y = remove_pairs_with_a_missing(X, Y)
                correlation, pval = spearman(X, Y)
                tests.append([features[i],features[j],pval, correlation, not_na])
    results = pd.DataFrame(tests, columns = ['Feature_1','Feature_2','pval', 'Correlation', 'Not_NAs'])
    return results

In [8]:
start_time = time.time()
results = btest_corr(dataAll, featuresX, featuresY)
rho_excution_time = time.time() - start_time

In [9]:
rho_excution_time

10.688559770584106

In [10]:
results

,Feature_1,Feature_2,pval,Correlation,Not_NAs
0,X0,X0,0.000000e+00,1.000000,50
1,X0,X1,1.511519e-19,0.909559,49
2,X0,X2,6.554862e-20,0.909350,50
3,X0,X3,8.315076e-23,0.932087,50
4,X0,X4,2.588865e-22,0.931758,49
...,...,...,...,...,...
20095,Y97,Y98,1.445902e-23,0.937008,50
20096,Y97,Y99,5.556863e-23,0.933255,50
20097,Y98,Y98,0.000000e+00,1.000000,50
20098,Y98,Y99,4.070674e-24,0.940343,50


In [11]:
rho_excution_time

10.688559770584106

In [12]:
within_X = results[results["Feature_1"].isin(featuresX) ]
within_X = within_X[within_X["Feature_2"].isin(featuresX) ]
within_X = within_X[within_X["Feature_1"] != within_X["Feature_2"]]

In [13]:
within_X

,Feature_1,Feature_2,pval,Correlation,Not_NAs
1,X0,X1,1.511519e-19,0.909559,49
2,X0,X2,6.554862e-20,0.909350,50
3,X0,X3,8.315076e-23,0.932087,50
4,X0,X4,2.588865e-22,0.931758,49
5,X0,X5,3.526796e-23,0.934547,50
...,...,...,...,...,...
14642,X96,X98,1.325589e-22,0.930709,50
14643,X96,X99,6.324625e-21,0.918110,50
14745,X97,X98,1.696742e-19,0.905512,50
14746,X97,X99,6.324625e-21,0.918110,50


In [14]:
within_Y = results[results["Feature_1"].isin(featuresY)]
within_Y = within_Y[within_Y["Feature_2"].isin(featuresY)]
within_Y = within_Y[within_Y["Feature_1"] != within_Y["Feature_2"]]

In [15]:
within_Y

,Feature_1,Feature_2,pval,Correlation,Not_NAs
15051,Y0,Y1,2.876300e-19,0.903308,50
15052,Y0,Y2,8.391473e-23,0.932060,50
15053,Y0,Y3,1.296812e-26,0.953337,50
15054,Y0,Y4,9.458936e-22,0.924577,50
15055,Y0,Y5,1.211416e-18,0.897027,50
...,...,...,...,...,...
20092,Y96,Y98,1.445902e-23,0.937008,50
20093,Y96,Y99,5.526886e-24,0.939555,50
20095,Y97,Y98,1.445902e-23,0.937008,50
20096,Y97,Y99,5.556863e-23,0.933255,50


In [16]:
X_Y = results[~np.logical_or(results["Feature_1"].isin(dataY.index), results["Feature_2"].isin(dataX.index))]
#X_Y = X_Y[X_Y["Feature_2"].isin(dataY.index) ]

In [17]:
X_Y


,Feature_1,Feature_2,pval,Correlation,Not_NAs
100,X0,Y0,0.654930,-0.064777,50
101,X0,Y1,0.846105,-0.028155,50
102,X0,Y2,0.835488,-0.030124,50
103,X0,Y3,0.719011,-0.052165,50
104,X0,Y4,0.931868,-0.012404,50
...,...,...,...,...,...
15045,X99,Y95,0.358257,0.132703,50
15046,X99,Y96,0.150612,0.206299,50
15047,X99,Y97,0.320788,0.143307,50
15048,X99,Y98,0.163753,0.200000,50


In [18]:
rho_X_Y = pd.pivot(X_Y, index="Feature_1", columns = "Feature_2",values = 'Correlation') #Reshape from long to wide
rho_X = pd.pivot(within_X, index="Feature_1", columns = "Feature_2",values = 'Correlation') #Reshape from long to wide
rho_Y = pd.pivot(within_Y, index="Feature_1", columns = "Feature_2",values = 'Correlation') #Reshape from long to wide

In [19]:
rho_X_Y

Feature_2,Y0,Y1,Y10,Y11,Y12,Y13,Y14,Y15,Y16,Y17,...,Y90,Y91,Y92,Y93,Y94,Y95,Y96,Y97,Y98,Y99
Feature_1,,,,,,,,,,,,,,,,,,,,,
X0,-0.064777,-0.028155,-0.032880,0.028549,-0.008071,-0.045866,-0.010041,0.013192,0.002166,0.034849,...,0.005956,-0.081299,-0.072638,-0.075015,-0.124385,-0.060199,-0.068701,-0.081299,-0.072638,-0.101004
X1,-0.166641,-0.132381,-0.074402,0.001543,-0.091251,-0.115725,-0.077985,-0.059914,-0.071081,-0.019379,...,0.127439,0.014537,0.040030,0.042889,-0.020316,0.069628,-0.008864,0.035455,0.033337,0.002354
X10,-0.093670,-0.071618,-0.065859,-0.001920,-0.071850,-0.086811,-0.020427,0.004922,-0.028303,-0.002313,...,0.104007,0.000591,0.004528,0.008712,-0.042331,0.028155,-0.024606,-0.009646,0.010827,-0.020033
X11,-0.140136,-0.079888,-0.078559,-0.010189,-0.053740,-0.104134,-0.060986,-0.038000,-0.068862,0.015407,...,0.022642,-0.068701,-0.056102,-0.057836,-0.101792,-0.042479,-0.068701,-0.077362,-0.062402,-0.102185
X12,-0.089048,-0.067877,-0.021654,0.049724,-0.054123,-0.058359,-0.035259,0.002249,-0.028186,0.035913,...,0.007453,-0.107201,-0.089578,-0.096815,-0.148073,-0.073749,-0.113476,-0.104534,-0.108037,-0.137505
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
X95,-0.082004,-0.055867,-0.035145,-0.088551,-0.134843,0.049409,-0.119659,-0.095245,-0.055621,-0.171097,...,0.240992,0.174803,0.262992,0.215397,0.207915,0.164206,0.237795,0.181102,0.225197,0.154361
X96,0.004627,0.018557,0.032979,-0.033816,-0.034055,0.112402,-0.051536,0.010681,0.031404,-0.096673,...,0.208703,0.162205,0.237795,0.185470,0.201614,0.109077,0.218898,0.155906,0.231496,0.140973
X97,-0.010730,-0.033422,0.025103,-0.060199,-0.080512,0.097441,-0.084613,-0.035785,-0.007580,-0.142351,...,0.214609,0.118110,0.218898,0.183894,0.189407,0.151998,0.187402,0.137008,0.193701,0.109470


In [20]:
rho_X

Feature_2,X1,X10,X11,X12,X13,X14,X15,X16,X17,X18,...,X90,X91,X92,X93,X94,X95,X96,X97,X98,X99
Feature_1,,,,,,,,,,,,,,,,,,,,,
X0,0.909559,0.909252,0.940846,0.905297,0.926685,0.925787,0.925787,0.906890,0.915650,0.940846,...,-0.008858,-0.021457,0.020276,-0.034055,0.032874,-0.002559,-0.008858,-0.046654,-0.021457,0.010039
X1,NaN,0.916645,0.926868,0.921808,0.930450,0.899101,0.936307,0.932568,0.938451,0.919181,...,0.063405,0.062150,0.075485,0.022695,0.104795,0.078047,0.051613,0.045286,0.047221,0.091016
X10,NaN,NaN,0.906890,0.917037,0.932725,0.928248,0.928248,0.911713,0.909350,0.918012,...,0.098228,0.060433,0.085630,0.043898,0.134449,0.075394,0.085630,0.043898,0.081693,0.117126
X11,NaN,NaN,NaN,0.909533,0.963473,0.934547,0.947146,0.921949,0.932087,0.919488,...,0.051772,0.013976,0.041535,-0.002559,0.079331,0.041535,0.032874,-0.006496,0.058071,0.047835
X12,NaN,NaN,NaN,NaN,0.933478,0.919495,0.929195,0.939575,0.903912,0.929875,...,0.043142,0.049025,0.031376,0.043848,0.095461,0.065575,0.028526,0.013413,0.052371,0.042750
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
X94,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,0.930709,0.905512,0.918110,0.918110,0.930709
X95,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,0.911811,0.930709,0.911811,0.943307
X96,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.911811,0.930709,0.918110


In [21]:
X_Y.pval

100      0.654930
101      0.846105
102      0.835488
103      0.719011
104      0.931868
           ...   
15045    0.358257
15046    0.150612
15047    0.320788
15048    0.163753
15049    0.285720
Name: pval, Length: 10000, dtype: float64

In [23]:
utils.fdr(X_Y["pval"].values)

array([1., 1., 1., ..., 1., 1., 1.])

In [28]:
X_Y_p_adust, X_Y_p_threshold = utils.bh(X_Y["pval"].values, .1)
within_X_p_adust, within_X_p_threshold = utils.bh(within_X["pval"].values, .1)
within_Y_p_adust, within_Y_p_threshold = utils.bh(within_Y["pval"].values, .1)


In [29]:
X_Y_p_threshold


0.6549299181254638

In [30]:
X_Y["pval"].values

array([0.65492992, 0.8461055 , 0.83548775, ..., 0.32078763, 0.16375308,
       0.28571983])

In [31]:
within_X_p_threshold

9.957213702277227e-16

In [32]:
within_Y_p_threshold

0.03441006979659323

In [33]:
X_Y_p_adust


array([1.31644205, 1.09443215, 1.10382845, ..., 2.1244214 , 3.28162487,
       2.27302965])

In [34]:
within_X_p_adust

array([4.82399577e-19, 2.20724942e-19, 6.59609428e-22, ...,
       5.33600704e-19, 2.48664757e-20, 5.33261910e-19])

In [35]:
within_Y_p_adust

array([1.54757459e-18, 1.34426515e-21, 1.33733717e-24, ...,
       3.09836146e-22, 9.65139400e-22, 1.19229812e-22])

In [36]:
X_Y["P_adusted"] = X_Y_p_adust
within_X["P_adusted"] = within_X_p_adust
within_Y["P_adusted"] = within_Y_p_adust

/var/folders/ry/39y8fffn25q2jpw0b96hq5fh0000gn/T/ipykernel_50114/3638538245.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X_Y["P_adusted"] = X_Y_p_adust


In [37]:
X_Y
within_X
within_Y

,Feature_1,Feature_2,pval,Correlation,Not_NAs,P_adusted
15051,Y0,Y1,2.876300e-19,0.903308,50,1.547575e-18
15052,Y0,Y2,8.391473e-23,0.932060,50,1.344265e-21
15053,Y0,Y3,1.296812e-26,0.953337,50,1.337337e-24
15054,Y0,Y4,9.458936e-22,0.924577,50,1.054544e-20
15055,Y0,Y5,1.211416e-18,0.897027,50,5.788136e-18
...,...,...,...,...,...,...
20092,Y96,Y98,1.445902e-23,0.937008,50,3.111833e-22
20093,Y96,Y99,5.526886e-24,0.939555,50,1.455217e-22
20095,Y97,Y98,1.445902e-23,0.937008,50,3.098361e-22
20096,Y97,Y99,5.556863e-23,0.933255,50,9.651394e-22


In [38]:
X_Y["bh_fdr_threshold"] = X_Y_p_threshold
within_X["bh_fdr_threshold"] = within_X_p_threshold
within_Y["bh_fdr_threshold"] = within_Y_p_threshold



/var/folders/ry/39y8fffn25q2jpw0b96hq5fh0000gn/T/ipykernel_50114/2076899948.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X_Y["bh_fdr_threshold"] = X_Y_p_threshold


In [39]:
X_Y = X_Y.sort_values(['pval', 'Correlation'],
                     ascending = [True, False])
within_X = within_X.sort_values(['pval', 'Correlation'],
                      ascending = [True, False])
within_Y = within_Y.sort_values(['pval', 'Correlation'],
                                ascending = [True, False])

In [40]:
X_Y

,Feature_1,Feature_2,pval,Correlation,Not_NAs,P_adusted,bh_fdr_threshold
365,X1,Y66,0.021416,0.327972,49,214.155016,0.65493
374,X1,Y75,0.033062,0.305052,49,165.311602,0.65493
5196,X27,Y74,0.036443,0.296654,50,121.477314,0.65493
373,X1,Y74,0.040235,0.294122,49,100.588520,0.65493
3021,X15,Y41,0.042550,0.287992,50,85.100808,0.65493
...,...,...,...,...,...,...,...
12544,X77,Y47,0.999459,-0.000098,50,0.999859,0.65493
4340,X22,Y93,0.999729,0.000049,50,1.000029,0.65493
9277,X53,Y8,0.999729,0.000049,50,0.999929,0.65493
13706,X87,Y34,0.999729,0.000049,50,0.999829,0.65493


In [41]:
outputpath = "/Users/rah/Documents/omicsEye/btest/demo/btest_output"
os.makedirs(outputpath, exist_ok=True)
X_Y.to_csv(outputpath + '/X_Y.tsv', sep="\t")
#rho_X = rho.iloc[0:500, 0:500]
#rho_X.to_csv(outputpath + '/simtable_X.tsv', sep="\t")
rho_X_Y.to_csv(outputpath + '/simtable.tsv', sep="\t")
within_X.to_csv(outputpath + '/within_X.tsv', sep="\t")
within_Y.to_csv(outputpath + '/within_Y.tsv', sep="\t")

In [2]:
#associations = blockplot.load_associations(path = outputpath + '/X_Y.tsv')
associations = blockplot.load_associations('/Users/rah/Dropbox/Ali-Docs/Research_docs/Projects/VastBiome/analysis/btest_bgc_mgx_replaceNA_FALSE_cov_25_measure_rpkg_mbx_pair_spearman_new/X_Y.tsv')

In [3]:
outputpath

NameError: name 'outputpath' is not defined

In [4]:
associations

[['616', ['GCF00020'], ['cystine'], '0.0', '1.0', '0.0'],
 ['623', ['GCF00020'], ['dihydroorotate'], '0.0', '1.0', '0.0'],
 ['673',
  ['GCF00020'],
  ['hydrochlorothiazide_C18-neg_v2'],
  '0.0',
  '1.0',
  '0.0'],
 ['3031', ['GCF00023'], ['C34:2 DAG'], '0.0', '1.0', '0.0'],
 ['3091', ['GCF00023'], ['C52:7 TAG'], '0.0', '1.0', '0.0'],
 ['3327', ['GCF00023'], ['tartarate'], '0.0', '1.0', '0.0'],
 ['5361', ['GCF00099'], ['2-aminoheptanoic acid'], '0.0', '1.0', '0.0'],
 ['5366', ['GCF00099'], ['2-hydroxyglutarate'], '0.0', '1.0', '0.0'],
 ['5388', ['GCF00099'], ['4-pyridoxate'], '0.0', '1.0', '0.0'],
 ['5414', ['GCF00099'], ['alanylalanine'], '0.0', '1.0', '0.0'],
 ['5420', ['GCF00099'], ['alpha-ketoglutarate'], '0.0', '1.0', '0.0'],
 ['5429', ['GCF00099'], ['arginine_HILIC-pos_v3'], '0.0', '1.0', '0.0'],
 ['5450', ['GCF00099'], ['C14 carnitine'], '0.0', '1.0', '0.0'],
 ['5455', ['GCF00099'], ['C14:1 carnitine'], '0.0', '1.0', '0.0'],
 ['5461', ['GCF00099'], ['C16:0 LPC_C8-pos_v2'], '0.0',

In [5]:
#simtable = blockplot.load_order_table(outputpath + '/simtable.tsv', associations)
simtable = blockplot.load_order_table('/Users/rah/Dropbox/Ali-Docs/Research_docs/Projects/VastBiome/analysis/btest_bgc_mgx_replaceNA_FALSE_cov_25_measure_rpkg_mbx_pair_spearman_new//simtable.tsv', associations)

In [46]:
X_Y

,Feature_1,Feature_2,pval,Correlation,Not_NAs,P_adusted,bh_fdr_threshold
365,X1,Y66,0.021416,0.327972,49,214.155016,0.65493
374,X1,Y75,0.033062,0.305052,49,165.311602,0.65493
5196,X27,Y74,0.036443,0.296654,50,121.477314,0.65493
373,X1,Y74,0.040235,0.294122,49,100.588520,0.65493
3021,X15,Y41,0.042550,0.287992,50,85.100808,0.65493
...,...,...,...,...,...,...,...
12544,X77,Y47,0.999459,-0.000098,50,0.999859,0.65493
4340,X22,Y93,0.999729,0.000049,50,1.000029,0.65493
9277,X53,Y8,0.999729,0.000049,50,0.999929,0.65493
13706,X87,Y34,0.999729,0.000049,50,0.999829,0.65493


In [47]:
blockplot.plot(
    simtable,
    associations,
    cmap="RdBu_r",
    mask=False,
    axlabels=["",""],
    outfile="/Users/rah/Documents/omicsEye/btest/demo/btest_output/blockplot.pdf",
    similarity="Spearman"
)